# Calculating Begin and End Days #

The notebook after Preprocessing focuses on calculating HS_begin_day, HS_end_day, HS_begin_day_c and HS_begin_day_c. These values are important for calculating the MHW metrics of Dc, Dp, Dws, HR, HRc etc.

In [ ]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

### Openning Data and Rounding ###
This is an important step in this notebook because it allows us to know what precision is being used when the computer is deciding what counts as a HotSpot above 0 and what does not. This is important because all our MHW values depend on knowing the first and last day where the HotSpot goes above 0 and so we need to know how this clasification is being made. The data that we are opening and using is the HotSpot data with the Heat Stress Year time series that we created in the last part of the Preprocessing notebook.

In [ ]:
HSY = xr.open_dataset('HSY_daily_cftime.nc')
arr = HSY.HotSpot

Here we are creating two variables that are essentially the same, except one has the actual HotSpot values for the gridcells with values above 0.00005 and 0 for the rest and the second one has the value of 1 for all gridcells classified as having values above 0. These two variables will be used as inputs to calculate other variables below.

In [ ]:
 arr_round = xr.where(arr < 0.00005, 0, arr)
arr_int = xr.where(arr < 0.00005 , 0, 1)

### Temporary Variables ###

For the next few cells we are creating variables that are necessary for calculating HS_begin_day_continous and HS_end_day_continous.I call these temporary variables because they are not going to be directly used for calculating the MHW metrics but are necessary as an intermediate step to get there. This is probably the hardest part to understand since there are a lot of different functions creating a lot of different variables, however I will try to explain step by step what each function is doing so it can be easier to follow.

#### Variable: arr_one ####
The first user defined function we are using is called 'one'. This function takes a DataArray as input and outputs another DataArray. The purpose of this function is to first apply a rolling sum to the data on the time dimension (center = True means that the result of the rolling sum will appear in the middle of the three values being considered) and see what values have a rolling sum of 0. The second part of the function is using the built in (.where) function to say that where the rolling sum results are 0, the output DataArray will have a value of 0, and where the three day rolling sum does not have a value of 0 the output DataArray will have a value of 1.

We then use one of the variables created abouve (arr_int) as the input for this function. However we want this to be applied to each year separately so we first use the groupby('time.year) function from Xarray and then use the .map() function which is a way of appling a function to a groupby object which we just created by grouping by the different years. Our output DataArray is called arr_one and it will be used later as the input to another function. 

In [ ]:
def one(x):
    return xr.where((x.rolling(time=3, center = True).sum().fillna(1)) == 0, 0, 1)

arr_one = arr_int.groupby('time.year').map(one)

#### Variable: Daily ####

Below we create the variable/DataArray "daily" by inputing arr_round into the user defined function "rolling_mean". What this function is doing is creating a rolling mean of the input DataArray given to it, which in this case is arr_round (the original HotSpot data rounded to 0.00005). We do this because HSpeak, one of the MHW variables, will be the maximum three_day rolling mean HotSpot value for each HSY. So we need to create a DataArray with the three day rolling mean values which is 'daily' below. We also start saving all our temporary values as NetCDF files. There are two reasons for this step, first if we run out of memory and need to restart the notebook we wont have to calculate them again and can just load them in, and second some of them will be used in another notebook when calculating the MHW metrics so we need to have their values saves. 

In [ ]:
def rolling_mean(x):
    return x.rolling(time=3, center=True).mean()

daily = arr_round.groupby('time.year').map(rolling_mean)
daily.to_netcdf('daily_00005_.nc')

#### Variable: max_mask ####

To create the variable/DataArray max_mask we input 'daily' into the user defined function (where_max). This function takes in a DataArray, finds the maximum value on the time dimension (maxi), and then applies a where funtion to create a new DataArray that has a value of 1 where the  input DataArray, 'daily' in this case, has its max value and 0 for all other values. Once again we groupby ('time.year') so that it is applied one year at a time and so for each year we have one max value for each gridcell. This mask variable will become important later when we are trying to figure out what continous HotSpot period includes HSpeak.  

In [ ]:
def where_max(x):
    maxi = x.max(dim = 'time')
    return xr.where(x == maxi, 1, 0)

max_mask = daily.groupby('time.year').map(where_max)
max_mask.to_netcdf('max_mask_00005_.nc')

#### Variable: max_mask_year ####

This variable is calculated by grouping max_mask by year and then applying the user defined function (max_loc) which uses the built in function argmax for the time dimension. This will output a data array with one value for each year (for each gridcell), where the value represents the day of the year in integers (from 1-365) where the max value occured. This temporary variable will be needed later when calculating  the metrics: HR and HRc.

In [ ]:
def max_loc(x):
    return x.argmax(dim = 'time')

max_mask_year = max_mask.groupby('time.year').map(max_loc)
max_mask_year.to_netcdf('max_mask_year.nc')

#### Variable: continuous_period ####

This might be the most difficult variable to understand. In simple terms what the two user defined functions below are doing is creating a DataArray that labels the different continuous HotSpot periods in a year so they can be differentiatied between. So the first continuous period will become 1,1,1,1, the second 2,2,2,2, the third 3,3,3,3 and so on and so forth. To accomplish this we use the scipy python package that has a special function for image processing called label that does what we need. However to be able to apply this function to multi-dimensional data such as ours we need to do a second step (which is the 'stacked' function below) where we groupby gridcells and then apply the scipy.ndimage.label function. So in the end if we work through the steps backwards, we are grouping our variable "arr_one" by year, then stacking the values of latitude and longitude, then grouping by the stacked gridcells, then applying the scipy.ndiamge.label function to each gridcell, then unstacking the gridcells so we can output a three dimensional DataArray with our labeled values. This step is necessary so we can find out which continuous period of HotSpots above 0 HSpeak is in, and then we can find the first and last day of that specific continuous period. However, creating the continuous period variable takes about 30 minutes for the sea surface temperature model data so it is important to save the results as a NetCDf file so that if we need to restart the notebook we do not have to run the code again.

In [ ]:
import scipy as sc
import scipy.ndimage

In [ ]:
def a_label(x): 
    label = (scipy.ndimage.label(x)[0])
    return np.array(label)

def stacked(x): 
    stack = x.stack(gridcell=['lat', 'lon'])
    return (stack.groupby("gridcell").map(a_label).unstack("gridcell"))

In [ ]:
continuous_period = arr_one.groupby('time.year').map(stacked)
continuous_period.to_netcdf('continous_period_august_4.nc')

#### Variable: max_group ####

In the cell below we are creating another mask variable that has the same amount of days and lattitude and longitude as our new continuous_period dataset but with only the continuous period value where max_mask is 1, and all other values are np.nan. This will mean that for each gridcell for each year there will only be one value abouve 0 for the max_group variable. This value will correspond to the continuous period label that contains max_mask (HSpeak), so if the value of max_mask that is 1 is in the third continuous period then max_group will only have one value in the location where max_mask is 1, which will be 3 and all others will be np.nan.

In [ ]:
max_group = continuous_period.where((max_mask == 1), 0)

#### Variables: filled & query_continuous ####

Below we use a user defined function (fill) to create the temporary variable 'filled' and then we use the variable 'filled' as an input to create the variable 'query_continuous'. 

What the fill function is doing is taking in the DataArray (max_group) which for each year and for each gridcell only has one value and all other values are nan, and returning a DataArray that fills each nan with the only non nan value for that year. So for example if max_group for the year 1985 and gridcell lat: 60 lon: 2, has a value of 3 where HSpeak occurs and all other values are nan, the filled function will return a DataArray with the value 3 for all 365 days of that year for that gridcell. So it would go from looking like [nan, nan, nan, nan, 3, nan, nan, nan] to [3, 3, 3, 3, 3, 3, 3, 3]. 

We then use the variable we just created (filled) as a mask to create a DataArray (query_continuous) that only has the values for the continuous period that includes HSpeak and all other values are 0. So instead of there being many different continunous periods labeled like we have in our continuous_period variable, now we have only the one that contains HSpeak for each gridcell for each year.

In [ ]:
def fill(x):
    forward = x.ffill(dim = 'time')
    backward = forward.bfill(dim = 'time')
    return backward

filled = max_group.groupby('time.year').map(fill)
filled.to_netcdf('filled_august_4.nc')

query_continuous = xr.where(continuous_period == filled, continuous_period, 0)
query_continuous.to_netcdf('query_00005_.nc')

#### Variable: cumsum ####

The next step is to use the user defined function 'd_acc' to return a DataArray with the cumulative sum of the query_continuous variable for each year. The function starts the cumulative sum at the first value in the continuous period and ends at the last value of the continuous period, with all other values set to 0. This is done so that when we want to find the first and last day of this continuous period we just simply need to find the time index of the lowest value that is not 0 and the time index of the largest value. 

In [ ]:
#this specific cumsum function that stops when it encounters the first 0 value, was taken from a StackOverflow answer
def d_acc(acc):
    return acc.cumsum(dim='time')-acc.cumsum(dim='time').where(acc.values == 0).ffill(dim='time').fillna(0)

cumsum = query_continuous.groupby('time.year').map(d_acc)
cumsum.to_netcdf('cumsum_00005_.nc')

#### Variable: arr_three ####

This variable is created specificly for the calculation of HS_begin and HS_end day. There is a rule for what can be considered the first and last HS day of the year. For a day to be considered as HS_begin_day there needs to be three continuous days of positive HotSpots (HS_begin day, and two more days after) and for a day to be considered the HS_end_day there need to be three consecutive days of positive HotSpot as well (two days prior and the HS_end_day). 

So to take this rule into account I created two user defined functions, the first is called 'positive_three'. This function takes in a DataArray does a rolling sum of three days and if the result is 3 it assigns a value of 1 and any other sum result gets assigned a value of 0. The input DataArray given to this function is arr_int that we created in the begining of this notebook and the output DataArray is called 'arr_three'. arr_three is a mask that contains 1s and 0s where the 1s represent values where a rolling sum of three days results in a value of three, or in other words there are three consecutive days of positive HotSpots. 

In [ ]:
def positive_three(x):
    return xr.where((x.rolling(time=3, center = True).sum().fillna(0)) == 3, 1, 0)

arr_three = arr_int.groupby('time.year').map(positive_three)

#### Variable: arr_one_back ####

The last temporary variable we create is arr_one_back, and we use the user defined function 'back_fill'. This function takes the variable arr_three created above as an input, and alters it in a couple ways. First it turns all the 0 values into nan. Then it fills two nans before a value of 1 with 1, and then two nan values that come after a value of 1 with 1.

It is hard to explain what we are doing here but basically when we apply the positive_three function above to check which first and last HS days follow the 3 day positive HS rule, we are setting some values to 0 that are actually positive HS values, they just dont have a rolling sum of 3. So the back_fill function fixes this for us so that we can then find the first and last HS day of the year correctly.

To illustrate what is happening imagine we have a DataArray where 1s represent days where HS is above 0 and 0 represents days where it is not:

[0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0 ,0, 0]

After applying the positive three function it would become:

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] 
(the ones are where there are three consecutive positive days, but we want the first 0 before the first positive day, and the first 0 after the last positive day of these three consecutive ones) 

So when we apply the back_fill function it becomes:

[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]

If you compare the first and last list, you can see that the first 1 in the last list corresponds to the 0 before the first 1 (that is a part of a three day continuous period of 1s) in the first list. The same is true for the last 1. So now we can find the location of the first and last 1 in the last list and we have the first and last HS_day that follows the three day continuous HS rule.

In [ ]:
def back_fill(x):
    make_nan = xr.where(x == 0, np.nan, x)
    backward = make_nan.bfill(dim = 'time', limit = 2)
    forward = backward.ffill(dim = 'time', limit = 2)
    return forward.fillna(0)

arr_one_back = arr_three.groupby('time.year').map(back_fill)

## HS_begin_day_c & HS_last_day_c ##

Finally we can find the first and last day of the continuous period that contains HSpeak. We do this with the two user defined functions below (first & last) and the input for both of them is the 'cumsum' temporary variable that we created above. What the first and last functions are doing is creating a yearly DataArray that has the value of the time index of the lowest value of cumsum that is not 0 for each gridcell, this is the begin day. And another DataArray that has the time index of the largest value in cumsum for each gridcell which will be last day. 

In [ ]:
def first(x):
    return xr.where(x>0, x, np.inf).argmin(dim = 'time')

def last(x):
    return xr.where(x>0, x, 0).argmax(dim = 'time')

last = cumsum.groupby('time.year').map(last)
first = cumsum.groupby('time.year').map(first)

last.to_netcdf('last_HS_c_raw.nc')
first.to_netcdf('first_HS_C_raw.nc')

## HS_begin_day & HS_end_day ##

To calculate HS_begin and HS_end day we use the two user defined functions below ('d_first' and 'd_last'). The input DataArray for both of these functions is the variable 'arr_one_back', (d_first) takes in arr_one_back and applies a cumulative sum to the values over the time dimension, we label this cumulative sum as acc. Then we use the xr.where function to replace all values in acc < 0 with np.inf. We can then apply the argmin function on the time dimension once again to retrieve the time index of the lowest value of the input DataArray for each gridcell for each year. This will give us the time index of the 0 before the first day the HS is above 0 for three consecutive days and is the variable first_HS. As for the function to find HS_end_day (d_last), we just have to apply a cumulative sum to the input DataArray (arr_one_back) and then apply the function argmax to get the time index of the last day of HS above 0 for the year. We dont have to worry about the fact that the cummulative sum will keep going after it encounters a 0, because argmax returns the first time index where the largest value occurs, which will be the last 1 in the arr_one_back data array. The variable 'last_HS' is a DataArray with yearly values for each gridcell of the day of the year that HS_end_day occurs.

In [ ]:
def d_first(x):
    acc = x.cumsum(dim = 'time')
    return xr.where(acc>0, acc, np.inf).argmin(dim = 'time')

first_HS = arr_one_back.groupby('time.year').map(d_first)


def d_last(x):
    acc = x.cumsum(dim = 'time')
    return acc.argmax(dim = 'time')

last_HS = arr_one_back.groupby('time.year').map(d_last)

#### Rules ####
There are certain rules that we need to apply to HS_begin and HS_end day before we save them to NetCDF files. A new limit/condition we set is that if HS_begin_day is 362 (when leap years are included) or 363 (when leap years are not included) then we should set the value of HS_begin_day to 0. Another limit is that if HS_last_day is 3, we assign a value of 0 to HS_last_day. Additionally, if HS_begin_day is 0 then HS_end_day should also be 0 and vice versa. These rules can be a little tricky to implement in Python because the indexes start at 0, so an HS_begin_day of 0 means that it is the first day of the year, therefore we need to create a variable where if HS_begin_day is 0 we give it a value of 1. This is necessary because when we are using HS_begin_Day as a condition to set HS_end_Day to 0, we dont want HS_end_day to be set to 0 if HS_begin_day is the first day of the year, only if HS_begin_day does not exist, due to some other rule. That is why we create the variable "first_HS_p1" below which can be seen in  the creating of the variable "HS_last".

In [ ]:
first_HS_p1 = xr.where(first_HS == 0, first_HS + 1, first_HS)
first_HS_end = xr.where(first_HS >= 361, 0, first_HS)
first_HS_end_p1 = xr.where(first_HS_p1 >= 361, 0, first_HS_p1)

last_HS_start = xr.where(last_HS <= 3, 0, last_HS)

HS_first = xr.where(last_HS_start == 0, 0, first_HS_end)
HS_first.to_netcdf('HS_first_raw.nc')

HS_last = xr.where(first_HS_end_p1 == 0, 0, last_HS_start)
HS_last.to_netcdf('HS_last_raw.nc')